In [ ]:
# default_exp play01


In [ ]:
#export
# spread jax work over 4 virtual cpu cores:
import os
import multiprocessing
cpu_count=multiprocessing.cpu_count()
print("jax will spread work to {} cpus".format(cpu_count))
os.environ["XLA_FLAGS"]="--xla_force_host_platform_device_count={}".format(cpu_count)


In [ ]:
#export
import matplotlib
import matplotlib.pyplot as plt
plt.ion()
import time
import numpy as np
import pandas as pd
import jax
import matplotlib
import argparse
import jax.numpy as jnp
import jax.config
import jax.scipy.optimize
import jax.experimental.maps
import jaxopt
import numpy.random
from matplotlib.pyplot import plot, imshow, tight_layout, xlabel, ylabel, title, subplot, subplot2grid, grid, legend, figure, gcf, xlim, ylim
from jax.experimental.maps import xmap


In [ ]:
#export
jax.config.update("jax_enable_x64", True)
_code_git_version="bdcbc4696db16e38cefae324f6b415cd15b111ab"
_code_repository="https://github.com/plops/cl-py-generator/tree/master//example/72_jaxopt/source/01_play.ipynb"
_code_generation_time="23:33:35 of Thursday, 2024-05-09 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
#export
df_status=pd.DataFrame([dict(name="numpy", version=np.__version__), dict(name="pandas", version=pd.__version__), dict(name="jax", version=jax.__version__), dict(name="matplotlib", version=matplotlib.__version__)])
print(df_status)


In [ ]:
#export
x=np.linspace(0, 4, 120)
y0=np.sin(x)
y=((y0)+(np.random.normal(loc=(0.    ), scale=(0.10    ), size=len(x))))


In [ ]:
#export
# try minimize many replicas with different noise with jax and xmap for named axes
n_batch=((4)*(10))
def make2d(a):
    return a.repeat(n_batch).reshape(-1, n_batch)
nx=120
x=np.linspace(0, 4, nx)
x_2d=make2d(x)
y0=np.sin(x)
y0_2d=make2d(y0)
noise_2d=np.random.normal(loc=(0.    ), scale=(0.10    ), size=y0_2d.shape)
y_2d=((y0_2d)+(noise_2d))
x0=jnp.array([(0.10    ), (1.0    ), (0.    )])
x0_2d=make2d(x0)
res=[]
def merit3(params, y, x):
    phase=params[0]
    amplitude=params[1]
    offset=params[2]
    return jnp.sum(((((y)-(((offset)+(((amplitude)*(jnp.sin(((phase)+(x))))))))))**(2)))
mesh_devices=np.array(jax.devices())
mesh_def=(mesh_devices,("x",),)
distr_jv=xmap(lambda x0, y, x: jax.scipy.optimize.minimize(merit3, x0, args=(y,x,), method="BFGS"), in_axes=({(1):("left")},{(1):("left")},{},), out_axes=["left", ...], axis_resources=dict(left="x"))
print("initialize jit")
jv0_start=time.time()
with jax.experimental.maps.mesh(*mesh_def):
    sol0=distr_jv(x0_2d, y_2d, x)
jv0_end=time.time()
print("call jit second time")
jv1_start=time.time()
with jax.experimental.maps.mesh(*mesh_def):
    sol=distr_jv(x0_2d, y_2d, x)
jv1_end=time.time()
d=dict(duration_jit_and_call=((jv0_end)-(jv0_start)), duration_call=((jv1_end)-(jv1_start)))
d["phase"]=sol.x[:,0].mean().item()
d["phase_err"]=sol.x[:,0].std().item()
phase=d["phase"]
d["phase0"]=x0[0].item()
d["amplitude"]=sol.x[:,1].mean().item()
d["amplitude_err"]=sol.x[:,1].std().item()
amplitude=d["amplitude"]
d["amplitude0"]=x0[1].item()
d["offset"]=sol.x[:,2].mean().item()
d["offset_err"]=sol.x[:,2].std().item()
offset=d["offset"]
d["offset0"]=x0[2].item()
d["success"]=sol.success[:].mean().item()
d["success_err"]=sol.success[:].std().item()
d["status"]=sol.status[:].mean().item()
d["status_err"]=sol.status[:].std().item()
d["fun"]=sol.fun[:].mean().item()
d["fun_err"]=sol.fun[:].std().item()
d["nfev"]=sol.nfev[:].mean().item()
d["nfev_err"]=sol.nfev[:].std().item()
d["njev"]=sol.njev[:].mean().item()
d["njev_err"]=sol.njev[:].std().item()
d["nit"]=sol.nit[:].mean().item()
d["nit_err"]=sol.nit[:].std().item()
res.append(d)
df2=pd.DataFrame(res)
print(df2.iloc[0])
figure(figsize=[12, 8])
pl=[2, 1]
subplot2grid(pl, [0, 0])
plot(x, y0, label="noise-free data")
for i in range(n_batch):
    plt.scatter(x, y_2d[:,i], label=("data with noise") if (((i)==(0))) else ("_no_legend_"), alpha=(9.00e-2), color="r")
plot(x, ((offset)+(((amplitude)*(jnp.sin(((phase)+(x))))))), label="fit avg")
for i in range(n_batch):
    phase=sol.x[i,0].mean().item()
    amplitude=sol.x[i,1].mean().item()
    offset=sol.x[i,2].mean().item()
    plot(x, ((offset)+(((amplitude)*(jnp.sin(((phase)+(x))))))), color="k", label=("fit") if (((i)==(0))) else ("_no_legend_"), alpha=(0.10    ))
legend()
grid()
subplot2grid(pl, [1, 0])
plot(x, ((y0)-(((offset)+(((amplitude)*(jnp.sin(((phase)+(x))))))))), label="residual noise-free data")
plt.scatter(x, ((y)-(((offset)+(((amplitude)*(jnp.sin(((phase)+(x))))))))), label="residual data with noise")
legend()
grid()
